In [15]:
import os
import openai
import sys
import panel as pn # GUI
import pandas as pd
import numpy as np
sys.path.append('../..')
import python_utils
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
tuned_model = os.environ['TUNED_MODEL']
# test = os.environ['OPENAI_API_KEY']
# print(test)

## Data Manipulation

In [16]:
df = pd.read_csv('products.csv')
df['inventory'] = np.random.randint(1, 10, df.shape[0])
df.head()

,product_id,product_name,aisle_id,department_id,inventory
0,1,Chocolate Sandwich Cookies,61,19,6
1,2,All-Seasons Salt,104,13,2
2,3,Robust Golden Unsweetened Oolong Tea,94,7,3
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,6
4,5,Green Chile Anytime Sauce,5,13,8


In [17]:
df.to_csv('products_new.csv', index=False)

## GPT Stuff

In [18]:
# def get_completion(prompt, model="gpt-3.5-turbo"):
def get_completion(prompt, model=tuned_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.Chatcompletion.create(
        model=model,
        messages=messages,
        temperature=0, #this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [19]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
    
    return pn.Column(*panels)

In [20]:
pn.extension()
panels = [] # collect display

context = [{'role':'system', 'content':"""
You are Phil, a chatbot for a grocery store called Fitty Groceries that containts a veriety of products.\
At the beginning of each interaction, you must introduce yourself as Phil and your purpose. \
After you first greet the customer, ask them how you can help them, then answer the questions regarding the inventory and products.\
Question should only involve those about the products we hold in our inventory.

"""}] # accumulate messages

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here...')
button_conversation = pn.widgets.Button(name="Submit")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here...')
    [1] Row
        [0] Button(name='Submit')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)